<a href="https://colab.research.google.com/github/SLCFLAB/hd2025_time_series/blob/main/day4/Day4_2_Transformer_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neuralforecast datasetsforecast

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from neuralforecast.losses.numpy import mae
from neuralforecast.core import NeuralForecast
from neuralforecast.models import Informer, Autoformer, PatchTST
from datasetsforecast.long_horizon import LongHorizon

### Data 전처리

In [ ]:
data_gold = pd.read_csv('https://raw.githubusercontent.com/SLCFLAB/hd2025_time_series/refs/heads/main/day4/dataset/gold.csv')
data_silver = pd.read_csv('https://raw.githubusercontent.com/SLCFLAB/hd2025_time_series/refs/heads/main/day4/dataset/silver.csv')

In [ ]:
def standard_scaler(data):
    mean = data['y'].mean()
    std = data['y'].std()
    data['y'] = (data['y']-mean)/std
    return mean, std, data

In [ ]:
mean_gold, std_gold, data_gold = standard_scaler(data_gold)
mean_silver, std_silver, data_silver = standard_scaler(data_silver)

In [ ]:
data = pd.concat([data_gold,data_silver])
data

### **학습**

In [ ]:
horizon = 1

In [ ]:
%%capture
models_tf = [Informer(h=horizon,                    # Forecasting horizon
                      input_size=10,                # Input size
                      max_steps=2000,               # Number of training iterations
                      val_check_steps=100,          # Compute validation loss every 100 steps
                      early_stop_patience_steps=3), # Stop training if validation loss does not improve
             Autoformer(h=horizon,
                        input_size=30,
                        max_steps=1000,
                        val_check_steps=100,
                        early_stop_patience_steps=3),
             PatchTST(h=horizon,
                        input_size=30,
                        max_steps=1000,
                        val_check_steps=100,
                        early_stop_patience_steps=3),
             ]

In [ ]:
tf = NeuralForecast(models=models_tf, freq=1)

data_tf = tf.cross_validation(df=data,
                               val_size=int(.1*len(data.ds.unique())),
                               test_size=int(.1*len(data.ds.unique())),
                               n_windows=None)

In [ ]:
data_tf = pd.read_csv('https://raw.githubusercontent.com/safeai-snu/TimeSeriesAI/refs/heads/main/ch3/dataset/gold_silver_pretrained.csv')

In [ ]:
tf_predict = data_tf[data_tf['ds']-data_tf['cutoff']==horizon]
tf_predict

In [ ]:
tf_gold = tf_predict[tf_predict['unique_id']=='gold']
tf_silver = tf_predict[tf_predict['unique_id']=='silver']

In [ ]:
tf_gold

### **결과 분석**

In [ ]:
tf_gold[['Informer','Autoformer','PatchTST','y']] = tf_gold[['Informer','Autoformer','PatchTST','y']]*std_gold + mean_gold
tf_silver[['Informer','Autoformer','PatchTST','y']] = tf_silver[['Informer','Autoformer','PatchTST','y']]*std_silver + mean_silver

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(tf_gold['ds'], tf_gold['y'], label='True')
plt.plot(tf_gold['ds'], tf_gold['Informer'], label='Informer')
plt.plot(tf_gold['ds'], tf_gold['Autoformer'], label='Autoformer')
plt.plot(tf_gold['ds'], tf_gold['PatchTST'], label='PatchTST')
plt.xlabel('Day')
plt.ylabel('Price')
plt.title('Gold Price Prediction')
plt.grid()
plt.legend()
plt.savefig('gold_tf_5step.png')
plt.show()

In [ ]:
mae_informer = mae(tf_gold['y'], tf_gold['Informer'])
mae_autoformer = mae(tf_gold['y'], tf_gold['Autoformer'])
mae_patchtst = mae(tf_gold['y'], tf_gold['PatchTST'])

print(f'Informer: {mae_informer:.3f}')
print(f'Autoformer: {mae_autoformer:.3f}')
print(f'PatchTST: {mae_patchtst:.3f}')

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(tf_silver['ds'], tf_silver['y'], label='True')
plt.plot(tf_silver['ds'], tf_silver['Informer'], label='Informer')
plt.plot(tf_silver['ds'], tf_silver['Autoformer'], label='Autoformer')
plt.plot(tf_silver['ds'], tf_silver['PatchTST'], label='PatchTST')
plt.xlabel('Day')
plt.ylabel('Price')
plt.title('Silver Price Prediction')
plt.grid()
plt.legend()
plt.savefig('silver_tf_5step.png')
plt.show()

In [ ]:
mae_informer = mae(tf_silver['y'], tf_silver['Informer'])
mae_autoformer = mae(tf_silver['y'], tf_silver['Autoformer'])
mae_patchtst = mae(tf_silver['y'], tf_silver['PatchTST'])

print(f'Informer: {mae_informer:.3f}')
print(f'Autoformer: {mae_autoformer:.3f}')
print(f'PatchTST: {mae_patchtst:.3f}')

### **AirPassengers Data**
Reference: https://github.com/Nixtla/neuralforecast/blob/main/nbs/examples/Getting_Started.ipynb

In [ ]:
from neuralforecast.utils import AirPassengersDF

Y_df = AirPassengersDF
Y_df.head()

In [ ]:
Y_df['y'].plot()

In [ ]:
%%capture
horizon = 12
models = [Informer(h=horizon,                 # Forecasting horizon
                input_size=2*horizon,           # Input size
                max_steps=500), # Stop training if validation loss does not improve
          Autoformer(h=horizon,
                input_size=2*horizon,
                max_steps=100),
          PatchTST(h=horizon,
                input_size=2*horizon,
                max_steps=100),
         ]

In [ ]:
nf = NeuralForecast(models=models, freq='M')

data_nf = nf.cross_validation(df=Y_df,
                               val_size=int(.1*len(Y_df.ds.unique())),
                               test_size=int(.1*len(Y_df.ds.unique())),
                               n_windows=None)

In [ ]:
nf_predict = data_nf[data_nf['cutoff']=='1959-12-31']

In [ ]:
nf_predict

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(Y_df['ds'], Y_df['y'], label='True')

for models in ['Informer','Autoformer','PatchTST']:
    plt.plot(nf_predict['ds'], nf_predict[models], label=models)
plt.xlabel('Month')
plt.ylabel('Passengers')
plt.title('AirPassengers Prediction')
plt.grid()
plt.legend()
plt.show()